In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

import timm
import torch
import torchvision
import torchsummary
from torch import nn
from torch.optim import Adam
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

## Code to edit the csv with correct image_file paths

image_data = pd.read_csv('/home/zephyr/Desktop/Newcastle_University/11_FP_D/Dataset/std_shape/training_1.csv')
image_data.head(5)

image_data.iloc[0][0][0:74]

image_data.iloc[0][0][74:]

a = '/home/zephyr/Desktop/Newcastle_University/11_FP_D'

for i in range(len(image_data)):
    image_data.iloc[i][0] = a + image_data.iloc[i][0][74:]

image_data.head(5)

image_data.to_csv('/home/zephyr/Desktop/Newcastle_University/11_FP_D/Dataset/std_shape/training_2.csv', index=False)

In [22]:
# loading the dataframe containing image paths and 1-hot encoded labels
image_data = pd.read_csv('/home/zephyr/Desktop/Newcastle_University/11_FP_D/Dataset/std_shape/training_2.csv')

In [23]:
image_data.head(5)

,image_path,labels
0,/home/zephyr/Desktop/Newcastle_University/11_F...,"[1, 0, 0]"
1,/home/zephyr/Desktop/Newcastle_University/11_F...,"[1, 0, 0]"
2,/home/zephyr/Desktop/Newcastle_University/11_F...,"[1, 0, 0]"
3,/home/zephyr/Desktop/Newcastle_University/11_F...,"[0, 0, 1]"
4,/home/zephyr/Desktop/Newcastle_University/11_F...,"[0, 1, 0]"


In [24]:
# code to convert labels top proper form (as string to list of integers - 1-hot encoded)
list(map(int, image_data.iloc[700][1].strip('][').split(', ')))

[0, 0, 1]

In [25]:
train, test = train_test_split(image_data, test_size=0.2, shuffle=True, random_state=7)

In [26]:
len(train)

3754

In [27]:
len(test)

939

In [2]:
model = timm.create_model('vit_base_patch32_224', pretrained = False, num_classes = 3)
#print(model(torch.randn(1, 3, 224, 224)))

In [3]:
torchsummary.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 768, 7, 7]       2,360,064
          Identity-2              [-1, 49, 768]               0
        PatchEmbed-3              [-1, 49, 768]               0
           Dropout-4              [-1, 50, 768]               0
         LayerNorm-5              [-1, 50, 768]           1,536
            Linear-6             [-1, 50, 2304]       1,771,776
           Dropout-7           [-1, 12, 50, 50]               0
            Linear-8              [-1, 50, 768]         590,592
           Dropout-9              [-1, 50, 768]               0
        Attention-10              [-1, 50, 768]               0
         Identity-11              [-1, 50, 768]               0
        LayerNorm-12              [-1, 50, 768]           1,536
           Linear-13             [-1, 50, 3072]       2,362,368
             GELU-14             [-1, 5

In [4]:
im_p = '/home/zephyr/Desktop/Newcastle_University/11_FP_D/code/436219-7159-48057-[1, 0, 0].png'
img = cv2.imread(im_p)
img = transforms.ToTensor()(img)
#img = img.unsqueeze(0)

In [5]:
writer = SummaryWriter('runs/vit_base_patch32_224_structure')

2022-06-21 13:41:16.922933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/zephyr/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-06-21 13:41:16.922997: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
writer.add_image('histo', img)

#!tensorboard --logdir=runs

In [7]:
writer.add_graph(model, img.unsqueeze(0))

#tensorboard --logdir=runs

/home/zephyr/.local/lib/python3.10/site-packages/torch/__init__.py:772: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
/home/zephyr/.local/lib/python3.10/site-packages/timm/models/vision_transformer.py:202: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)


In [8]:
writer.close()